##### IMPORTAMOS los datos de las windows

In [5]:
import numpy as np

data= np.load('../data/windows_data/chb01_raw_eeg_128_full.npz')
data= list(data['arr_0'])


AttributeError: 'NpzFile' object has no attribute 'shape'

##### IMPORTAMOS los metadatos de las windows, y le asignamos la window a la que le pertenece

In [3]:
import pandas as pd

metadata= pd.read_csv('../data/metadata/chb01_raw_eeg_128_full.csv')

In [6]:
data

In [12]:
#mean, std, kurtosis, skewness, median, entropy, min, max, power_energy
import scipy.stats as stats


#### TENEMOS QUE HACER LAS FEATURES PERO PARA CADA CANAL, CADA FEATURE TIENE QUE TENER 21 VALORES (21 CANALES)

def calculate_window_features(window_values, canales=21):
    for c in range(canales):
        col_canal= window_values[:,c]

        mean = np.mean(col_canal)
        std = np.std(col_canal)
        kurtosis = stats.kurtosis(col_canal)
        skewness = stats.skew(col_canal)
        median = np.median(col_canal)
        entropy= stats.entropy(col_canal)
        min_value= np.min(col_canal)
        max_value= np.max(col_canal)
        power_energy= np.sum(np.power((col_canal),2))

    return mean, std, median, kurtosis, skewness, entropy, min_value, max_value, power_energy
    

In [13]:
def load_non_seizure_windows(metadata, data):
    df_features = pd.DataFrame()

    non_seizure_metadata= metadata[metadata['label']== 0] #Que no haya ataque
    print('# non seizure windows:', non_seizure_metadata.shape[0])

    nrows= non_seizure_metadata.shape[0]
    label= 0 #no hay ataque

    for index in range(nrows):
        metadata_non_seizure_window= non_seizure_metadata.iloc[index]
        id_non_seizure_window= metadata_non_seizure_window['id']

        data_non_seizure_window= data[id_non_seizure_window]
        
        mean, std, median, kurtosis, skewness, entropy, min_value, max_value, power_energy= calculate_window_features(data_non_seizure_window)

        row = {"mean": mean, "std":std, "median":median, "kurtosis":kurtosis, "skewness":skewness, "entropy":entropy, "min_value":min_value, "max_value":max_value, "power_energy":power_energy, "label":label}

        df_features = df_features.append(row, ignore_index=True)

    return df_features #label = 0


df_features= load_non_seizure_windows(metadata, data)

rows = len(df_features.axes[0]) 
cols = len(df_features.axes[1]) 

print(rows, cols)

# non seizure windows: 10700


<ipython-input-13-5db56d97107b>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_features = df_features.append(row, ignore_index=True)
<ipython-input-13-5db56d97107b>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_features = df_features.append(row, ignore_index=True)
<ipython-input-13-5db56d97107b>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_features = df_features.append(row, ignore_index=True)
<ipython-input-13-5db56d97107b>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_features = df_features.append(row, ignore_index=True)
<ipython-input-13-5db56d97107b>:20: FutureWarning: The frame.append method is deprecated

10700 10


<ipython-input-13-5db56d97107b>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_features = df_features.append(row, ignore_index=True)


In [14]:
def load_seizure_periods(metadata, data):
    seizure_metadata= metadata[metadata['label']== 1] #Que haya ataque


    nrows= seizure_metadata.shape[0]
    id_prev= None

    list_seizure_atacks = []

    for index in range(nrows):
        metadata_seizure_window= seizure_metadata.iloc[index]
        id_seizure_window= metadata_seizure_window['id']

        data_seizure_window= data[id_seizure_window]

        if id_prev== None:
            data_window_prev = data_seizure_window
            id_prev = id_seizure_window

        elif id_prev == (id_seizure_window - 1): #if nrows=40 -> (5120, 21)
            data_window_prev= np.vstack((data_window_prev, data_seizure_window))
            id_prev = id_seizure_window

        elif id_prev!= (id_seizure_window - 1):
            list_seizure_atacks.append(data_window_prev)
            data_window_prev = data_seizure_window
            id_prev = id_seizure_window
    

        if index == (nrows-1):
            list_seizure_atacks.append(data_window_prev)

    return list_seizure_atacks


non_seizure_metadata= metadata[metadata['label']== 0]
list_seizure_atacks= load_seizure_periods(metadata, data)


In [15]:
def Windows_DataAumentation(list_seizure_atacks, k, df_features):
    index_inicial = 0
    index_final = 128
    label = 1

    number_windows= 0

    for seizure_period in list_seizure_atacks:
        while index_final < seizure_period.shape[0]:
            windows_data_aumentation= seizure_period[index_inicial: index_final, :]

            mean, std, median, kurtosis, skewness, entropy, min_value, max_value, power_energy= calculate_window_features(windows_data_aumentation)

            row = {"mean": mean, "std":std, "median":median, "kurtosis":kurtosis, "skewness":skewness, "entropy":entropy, "min_value":min_value, "max_value":max_value, "power_energy":power_energy, "label":label}

            df_features = df_features.append(row, ignore_index=True)


            index_inicial += k
            index_final += k
            number_windows+= 1


    return df_features, number_windows

df_features, number_windows_atack= Windows_DataAumentation(list_seizure_atacks, 3, df_features)

rows = len(df_features.axes[0]) 
cols = len(df_features.axes[1]) 

print(f'Pandas de las features (atacks + non_atacks):  {rows, cols}')


print('# seizure windows sin DATA AUMENTATION: ', metadata[metadata['label']== 1].shape[0])
print('# seizure windows con DATA AUMENTATION', number_windows_atack)

print('# non seizure windows:', non_seizure_metadata.shape[0])


#df_features: valores que se pasaran al SVM -> las 9 features con el label [X-> features, y->label]



<ipython-input-15-f0cbf2465ba4>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_features = df_features.append(row, ignore_index=True)
<ipython-input-15-f0cbf2465ba4>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_features = df_features.append(row, ignore_index=True)
<ipython-input-15-f0cbf2465ba4>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_features = df_features.append(row, ignore_index=True)
<ipython-input-15-f0cbf2465ba4>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_features = df_features.append(row, ignore_index=True)
<ipython-input-15-f0cbf2465ba4>:16: FutureWarning: The frame.append method is deprecated

Pandas de las features (atacks + non_atacks):  (12364, 10)
# seizure windows sin DATA AUMENTATION:  40
# seizure windows con DATA AUMENTATION 1664
# non seizure windows: 10700


<ipython-input-15-f0cbf2465ba4>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_features = df_features.append(row, ignore_index=True)
<ipython-input-15-f0cbf2465ba4>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_features = df_features.append(row, ignore_index=True)
<ipython-input-15-f0cbf2465ba4>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_features = df_features.append(row, ignore_index=True)
<ipython-input-15-f0cbf2465ba4>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_features = df_features.append(row, ignore_index=True)


In [16]:
import os

def save_features_data(metadata, filename, folder):
    """
    Guarda les windows i les metadades al folder

    :param folder: ruta del directori on guardarem la info
    :type folder: str
    """

    # if necessary create folder
    if not os.path.exists(folder):
        os.makedirs(folder)
    
    # save metadata as csv
    metadata.to_csv(os.path.join(folder, filename), index=False)


save_features_data(df_features, "features_data_chb01.csv", "features_data")

    